In [1]:
#pip install --upgrade tomotopy
#pip install --upgrade pandas
#pip install reshape
#pip install gensim

import re
import numpy as np
import tomotopy as tp
from kiwipiepy import Kiwi, Option
from pyvis.network import Network

In [2]:
kiwi = Kiwi()

In [3]:
Kiwi(num_workers=-0, model_path='./', options=Option.LOAD_DEFAULT_DICTIONARY | Option.INTEGRATE_ALLOMORPH)

In [4]:
try:
    corpus = tp.utils.Corpus.load('k.cps')
except IOError:
    kiwi = Kiwi()
    kiwi.add_user_word('전달력', tag='NNG')
    kiwi.add_user_word('영상미', tag='NNG')
    kiwi.add_user_word('영상기법', tag='NNG')
    kiwi.add_user_word('영상구도', tag='NNG')
    kiwi.add_user_word('배경음악', tag='NNG')
    kiwi.add_user_word('구독자수', tag='NNG')   
    kiwi.add_user_word('레고프렌즈', tag='NNG')
    kiwi.add_user_word('딱히', tag='MAG')
    kiwi.add_user_word('교육', tag='NNP')
    kiwi.add_user_word('방송', tag='NNP')
    kiwi.add_user_word('브이로그', tag='NNP')
    kiwi.add_user_word('생일', tag='NNP')
    kiwi.add_user_word('날', tag='NNP')
    kiwi.add_user_word('전반적', tag='NNP')
    kiwi.add_user_word('조회수', tag='NNP')
    kiwi.add_user_word('매니아층', tag='NNP')
    kiwi.add_user_word('생방송', tag='NNP')
    kiwi.add_user_word('먹방', tag='NNP')
    kiwi.add_user_word('첫영상', tag='NNP')
    kiwi.add_user_word('배경음악', tag='NNP')
    kiwi.add_user_word('맛집', tag='NNP')
    kiwi.add_user_word('재건축', tag='NNP')
    kiwi.add_user_word('언박싱', tag='NNP')    
    kiwi.prepare()
    pat_tag = re.compile('NN[GP]|V[VA]|MAG|MM') 

In [5]:
    def tokenizer(raw, user_data):
        res, _ = user_data()[0]
        for w, tag, start, l in res:
            if not pat_tag.match(tag) or len(w) <= 1: continue
            yield w + ('다' if tag.startswith('V') else ''), start, l

In [6]:
    corpus = tp.utils.Corpus(
        tokenizer=tokenizer
    )
    
    corpus.process((line, kiwi.async_analyze(line)) for line in open('for text mining_final_210309_copy2.txt', encoding='UTF-8'))
    corpus.save('data_210309.cps')

case1. tw기본값 (tw=tp.TermWeight.ONE)으로 세팅했을 때!

In [7]:
# 최소 1개 이상 문헌에 등장하고, 전체 출현빈도는 2 이상인 단어만 사용
# 그리고 상위 10개 고빈도 단어는 분석에서 제외하고싶으면 rm_top=10을 추가, 안하고 싶으면 0
# 주제 개수는 20개

#tw=용어 가중치(tp.TermWeight.ONE은 기본값으로 모든 용어의 가중치를 동일하게 간주)

mdl = tp.CTModel(tw=tp.TermWeight.ONE, min_df=1, min_cf=2, rm_top=0, k=20, corpus=corpus)
mdl.train(0)

case2. tw에 PMI로 가중치 부여했을 때

In [8]:
# 최소 1개 이상 문헌에 등장하고, 전체 출현빈도는 2 이상인 단어만 사용
# 그리고 상위 10개 고빈도 단어는 분석에서 제외하고싶으면 rm_top=10을 추가, 안하고 싶으면 0
# 주제 개수는 20개

#tw=용어 가중치(tp.TermWeight.ONE은 기본값으로 모든 용어의 가중치를 동일하게 간주)

mdl = tp.CTModel(tw=tp.TermWeight.PMI, min_df=1, min_cf=2, rm_top=0, k=20, corpus=corpus)
mdl.train(0)

case3. tw에 PMI로 가중치 부여하고, min_df=1, min_cf=1로 설정 (개발자분 피드백)

In [9]:
# 최소 0개 이상 문헌에 등장하고, 전체 출현빈도는 0 이상인 단어만 사용
# 그리고 상위 10개 고빈도 단어는 분석에서 제외하고싶으면 rm_top=10을 추가, 안하고 싶으면 0
# 주제 개수는 10개

#tw=용어 가중치(tp.TermWeight.ONE은 기본값으로 모든 용어의 가중치를 동일하게 간주)

mdl = tp.CTModel(tw=tp.TermWeight.PMI, min_df=0, min_cf=0, rm_top=0, k=10, corpus=corpus)
mdl.train(0)

In [10]:
#num_beta_sample은 만 개 이상이면 1~5, 수 천개라면 최소 10, 수 백개에 불과하면 20이상으로 키우기
mdl.num_beta_sample = 10
print('Num docs:{}, Total Words:{}'.format(
    len(mdl.docs), mdl.num_words
))

Num docs:147, Total Words:433


In [11]:
#깁스샘플링
for i in range(0, 500, 20):
    print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))
    mdl.train(20)

Iteration: 0000, LL per word: -28.31
Iteration: 0020, LL per word: -25.99
Iteration: 0040, LL per word: -27.23
Iteration: 0060, LL per word: -27.98
Iteration: 0080, LL per word: -28.7
Iteration: 0100, LL per word: -29.57
Iteration: 0120, LL per word: -29.58
Iteration: 0140, LL per word: -29.55
Iteration: 0160, LL per word: -30.11
Iteration: 0180, LL per word: -29.6
Iteration: 0200, LL per word: -30.49
Iteration: 0220, LL per word: -30.83
Iteration: 0240, LL per word: -29.91
Iteration: 0260, LL per word: -31.71
Iteration: 0280, LL per word: -29.91
Iteration: 0300, LL per word: -30.51
Iteration: 0320, LL per word: -29.85
Iteration: 0340, LL per word: -29.99
Iteration: 0360, LL per word: -29.52
Iteration: 0380, LL per word: -30.11
Iteration: 0400, LL per word: -31.01
Iteration: 0420, LL per word: -29.83
Iteration: 0440, LL per word: -30.33
Iteration: 0460, LL per word: -30.85
Iteration: 0480, LL per word: -30.8


In [12]:
print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))

Iteration: 0480, LL per word: -30.37


In [13]:
mdl.summary()

#Words: CTM 분석 후 추출된 문서들 전체의 단어의 개수 (문서 당 같은 단어가 2개 있을 수도 있고 중복 가능)
#Total Vocabs: 작은 빈도의 단어들을 제거한 뒤 남은 토픽모델링 후 어휘의 개수
#Used_Vocabs: 모델에 실제로 사용된 어휘만을 포함하는 dictionary 타입의 어휘 사전
#prior_cov : 토픽의 사전분포인 로지스틱 정규분포의 공분산 행렬
#prior_mean : 토픽의 사전분포인 로지스틱 정규분포의 평균 벡터
#k : 토픽의 개수, 1 ~ 32767 범위의 정수.
#alpha는 CTM 모형의 모수 
#eta는 모수 
#num.iterations는 반복 횟수로 z를 몇 번 추출하느냐


<Basic Info>
| CTModel (current version: 0.10.2)
| 147 docs, 433 words
| Total Vocabs: 268, Used Vocabs: 268
| Entropy of words: -5.36839
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 2
| Log-likelihood per word: -30.36730
|
<Initial Parameters>
| tw: TermWeight.PMI
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 10 (the number of topics between 1 ~ 32767)
| smoothing_alpha: 0.1 (small smoothing value for preventing topic counts to be zero)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 1489786347 (random seed)
| trained in version 0.10.2
|
<Parameters>
| prior_mean (Prior mean of Logit-normal for the per-document topic distributions)
|  [-3.883528  -3.037404  -4.2298903 -3.4846911 -4.185564  -3.7446342
|   -3.2632713 -3.5483396 -3.296707  -2.4842188]
| prior_cov (Prior covariance of Logit-n

In [14]:
mdl.used_vocabs   

#Used Vocabs 의미 >> 78개
#242 words >> k(20)에 속한 총 단어의 수 (84개가 중복되어서 들어감)  == 총 단어의 빈도 수

['브이로그', '일상', '게임', '편집', '좋아하다', '사람', '재미있다', '정보', '주제', '여행', '만들다', '자유', '소통', '공유', '많이', '모습', '강아지', '첫영상', '활동', '도움', '생활', '라이브', '추억', '운동', '취미', '교육', '귀엽다', '패션', '분야', '대하다', '가장', '처음', '시청자', '실시간', '제작', '즐거움', '음악', '관심', '내용', '콘텐츠', '먹방', '업로드', '촬영', '자연', '영상미', '보이다', '고양이', '수업', '아이', '직장', '즐기다', '함께', '노력', '직접', '연주', '노래', '지식', '전달', '기존', '리뷰', '위하다', '방송', '사회', '경험', '기록', '어리다', '공감', '나오다', '가족', '요리', '배경음악', '신경', '요소', '제품', '비교', '올리다', '사용', '다이어트', '새롭다', '메뉴', '딱히', '구독자수', '얽매이다', '저장', '건강', '차량', '시승', '전달력', '메추리', '실력', '다양', '접근성', '용이', '남기다', '온라인', '헬스장', '다른', '동료', '동호회', '구입', '골프', '연습', '방법', '반려', '환경', '정성', '들어가다', '자녀', '역사', '컨셉', '생각', '그저', '그대로', '산책', '해외', '소개', '패션쇼', '유명한', '밴드', '신규', '접하다', '겨울왕국', '라방', '특징', '후원', '채팅', '식물', '키우다', '실무', '동기', '동창', '리액션', '공감대', '다시', '지금', '학교', '과제', '시연', '스피드', '페인팅', '반려견', '제공', '주식', '토크', '경제', '유투', '육아', '베이비', '특정', '자신', '특별', '매니아층', '토킹', '편하다', '과외', '영화', '채널'

In [15]:
#Visualize the result

g = Network(width=800, height=800, font_color="#333")
correl = mdl.get_correlations().reshape([-1])

In [16]:
correl.sort()

In [17]:
# 상관계수 상위 10%만 간선으로 잇고
top_tenth = mdl.k * (mdl.k - 1) // 10
top_tenth = correl[-mdl.k - top_tenth]

In [18]:
topic_counts = mdl.get_count_by_topics()

In [19]:
print(topic_counts)

#합하면 242 (242words) >> 빈도수 242

[39 49 41 44 37 28 59 43 45 48]


In [20]:
for k in range(mdl.k):
    label = "#{}".format(k)
    title= ' '.join(word for word, _ in mdl.get_topic_words(k, top_n=8))
    print('Topic', label, title)
    label += '\n' + ' '.join(word for word, _ in mdl.get_topic_words(k, top_n=3))
    g.add_node(k, label=label, title=title, shape='ellipse', value=float(topic_counts[k]))
    for l, correlation in zip(range(k - 1), mdl.get_correlations(k)):
        if correlation < top_tenth: continue
        g.add_edge(k, l, value=float(correlation), title='{:.02}'.format(correlation))

Topic #0 게임 많이 즐기다 콘텐츠 기존 신경 힘들다 최대한
Topic #1 소통 강아지 주제 음악 귀엽다 실시간 자연 나오다
Topic #2 추억 첫영상 라이브 생활 즐거움 내용 관심 고양이
Topic #3 좋아하다 정보 교육 어리다 촬영 보이다 직장 노력
Topic #4 여행 운동 모습 제작 수업 위하다 눈싸움 반려
Topic #5 사람 자유 활동 먹방 사용 창단 다른 메뉴
Topic #6 만들다 도움 분야 처음 대하다 영상미 가장 기록
Topic #7 재미있다 편집 공유 공감 방송 리뷰 요소 함께
Topic #8 브이로그 일상 패션 아이 편안 산책 스타일 장난감
Topic #9 취미 직접 연주 노래 가족 제품 비교 전달력


In [21]:
#토픽 topic_id에 속하는 상위 top_n개의 단어와 각각의 확률을 반환.
#반환 타입은 (단어:str, 확률:float) 튜플의 list형
#확률은 현재 토픽 내 각각의 단어들의 발생확률?

topic_words = mdl.get_topic_words(k, top_n=50)

In [22]:
import pandas as pd

In [23]:
df_topic_words = pd.DataFrame(topic_words)

In [24]:
#https://dandyrilla.github.io/2017-08-12/pandas-10min/

df_topic_words

,0,1
0,취미,0.047285
1,직접,0.043348
2,연주,0.040240
3,노래,0.038422
4,가족,0.037132
5,제품,0.034023
6,비교,0.034023
7,전달력,0.027268
8,아직,0.027268
9,구입,0.027268


In [25]:
df_topic_words.describe()

,1
count,50.000000
mean,0.019804
std,0.012046
min,0.000045
25%,0.017943
50%,0.020051
75%,0.024160
max,0.047285


In [26]:
topic_words

[('취미', 0.047284699976444244),
 ('직접', 0.043348412960767746),
 ('연주', 0.0402400903403759),
 ('노래', 0.038421835750341415),
 ('가족', 0.037131763994693756),
 ('제품', 0.03402344137430191),
 ('비교', 0.03402344137430191),
 ('전달력', 0.027268242090940475),
 ('아직', 0.027268242090940475),
 ('구입', 0.027268242090940475),
 ('모든', 0.027268242090940475),
 ('공감대', 0.027268242090940475),
 ('겨울왕국', 0.02415991760790348),
 ('레고프렌즈', 0.02415991760790348),
 ('실체', 0.02415991760790348),
 ('실력', 0.02415991760790348),
 ('부르다', 0.022341664880514145),
 ('홍보', 0.022341664880514145),
 ('음식', 0.022341664880514145),
 ('아기', 0.022341664880514145),
 ('문화', 0.022341664880514145),
 ('언어', 0.022341664880514145),
 ('방법', 0.021051593124866486),
 ('연습', 0.021051593124866486),
 ('이루어지다', 0.020050937309861183),
 ('정엽', 0.020050937309861183),
 ('언덕', 0.020050937309861183),
 ('철거', 0.020050937309861183),
 ('마지막', 0.020050937309861183),
 ('이별', 0.020050937309861183),
 ('여름', 0.020050937309861183),
 ('현장', 0.020050937309861183),
 ('쿠

In [27]:
g.barnes_hut(gravity=-1000, spring_length=20)

In [28]:
g.show_buttons()
g.show("topic_network6.html")

In [29]:
import pyLDAvis
import pyLDAvis.gensim

/Users/oridori/git/python/oridori/venv/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Users/oridori/git/python/oridori/venv/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:169: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-no

In [30]:
pyLDAvis.enable_notebook()

In [31]:
topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])

In [32]:
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])

In [33]:
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])

In [34]:
vocab = list(mdl.used_vocabs)

In [35]:
term_frequency = mdl.used_vocab_freq

In [36]:
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims= True)

In [37]:
prepared_data = pyLDAvis.prepare(
    topic_term_dists,
    doc_topic_dists,
    doc_lengths,
    vocab,
    term_frequency
)

In [41]:
pyLDAvis.display(prepared_data)